# SQL

## Project goal

The coronavirus has taken the world by surprise, changing the usual order of things. In their free time, city residents no longer go out, do not visit cafes and shopping centers. But there is more time for books. Startupers noticed this and rushed to create applications for those who like to read.

Your company decided to be on the wave and bought a large service for reading books by subscription. Your first task as an analyst is to analyze the database.
It contains information about books, publishers, authors, as well as user reviews of books. This data will help formulate a value proposition for a new product.

## Database connection

In [1]:
# import all packages
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# parameter setting
db_config = {'user': 'praktikum_student', 
              'pwd': 'Sdf4$2;d-d30pp', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # 
               'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db']) 
# save connector
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Exploring tables

### Data Description

**Table `books`**

Contains information about books:

- `book_id` — book identifier;
- `author_id` — author identifier;
- `title` — title of the book;
- `num_pages` - number of pages;
- `publication_date` - publication date of the book;
- `publisher_id` - publisher identifier.

**The `authors` table**

Contains information about the authors:

- `author_id` — author identifier;
- `author` is the name of the author.

**Table `publishers`**

Contains information about publishers:

- `publisher_id` — publisher identifier;
- `publisher` — publisher name;

**Table `ratings`**

Contains data about user ratings of books:

- `rating_id` — rating identifier;
- `book_id` — book identifier;
- `username` - the name of the user who left the rating;
- `rating` - rating of the book.

**Table `reviews`**

Contains data about user reviews of books:

- `review_id` — review identifier;
- `book_id` — book identifier;
- `username` - the name of the user who wrote the review;
- `text` — review text.

![](https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=2000&userId=d635827d-e7fd-4206-9d2a-00f59cd9d0eb&cache=v2)

### Database table queries

#### Table 'books'

In [3]:
query = '''
    SELECT *
    FROM books
    LIMIT 5
    '''
pd.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


#### Table 'authors'

In [4]:
query = '''
    SELECT *
    FROM authors
    LIMIT 5
    '''
pd.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Table  'ratings'

In [5]:
query = '''
    SELECT *
    FROM ratings
    LIMIT 5
    '''
pd.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### Table 'reviews'

In [6]:
query = '''
    SELECT *
    FROM reviews
    LIMIT 5
    '''
pd.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Table 'publishers'

In [7]:
query = '''
    SELECT *
    FROM publishers
    LIMIT 5
    '''
pd.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Study

### Let's count how many books came out after January 1, 2000

In [8]:
query = '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01';
'''
pd.read_sql(query, con = engine)

,count
0,819


### For each book, we calculate the number of reviews and the average rating

In [9]:
query = '''
    SELECT b.book_id, 
        b.title,
        COUNT(DISTINCT rv.review_id) AS count_review,
        ROUND(AVG(rt.rating), 2) AS avg_rating
    FROM books AS b
    LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
    LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
    GROUP BY b.book_id
    ORDER BY count_review DESC, avg_rating DESC;

'''
pd.read_sql(query, con = engine)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


### Let's determine the publisher that has released the largest number of books thicker than 50 pages (thereby excluding brochures from the analysis)

In [10]:
query = '''
    SELECT p.publisher,
        COUNT(b.book_id) AS count_book
    FROM books AS b
    LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY count_book DESC
    LIMIT 1;
'''
pd.read_sql(query, con = engine)

,publisher,count_book
0,Penguin Books,42


### Let's determine the author with the highest average rating of books (we will take into account only books with 50 or more ratings)

In [11]:
query = '''
    SELECT a.author,
        ROUND(AVG(r.rating), 2) AS avg_rating
    FROM books AS b
    LEFT JOIN authors AS a ON b.author_id = a.author_id
    LEFT JOIN ratings AS r ON b.book_id = r.book_id
    WHERE b.book_id IN (SELECT book_id
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating_id) > 50)
    GROUP BY a.author
    ORDER BY avg_rating DESC
    LIMIT 1;
'''
pd.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


### Let's calculate the average number of reviews from users who have given more than 50 ratings

In [12]:
query = '''
    SELECT ROUND(AVG(count_review))
     
    FROM (SELECT username,
                 COUNT(review_id) AS count_review
         FROM reviews
         WHERE username IN (SELECT username
                           FROM ratings
                           GROUP BY username
                           HAVING COUNT(rating_id) > 50)
         GROUP BY username) as count;       
'''
pd.read_sql(query, con = engine)

,round
0,24.0


## General conclusions

During the study of database containing information about books, publishers, authors, as well as user reviews of books, the following conclusions were obtained (on the basis of which a value proposition for a new product will be formulated in the future):
1. Since January 1, 2000, 819 books have been released.
2. For each book, the number of reviews and the average rating were calculated: the maximum number of reviews for the book **Twilight (Twilight #1)** - 7 reviews and the average rating - 3.66.
3. **Penguin Books** published 42 books thicker than 50 pages (that is, excluding brochures), which is the maximum value among all publishers.
4. Author **J.K. Rowling/Mary GrandPré** received the highest average book rating of 4.3 (only books with 50 or more ratings were counted).
5. The average number of reviews from users who gave more than 50 ratings was 24 reviews.